In [13]:
import os
import json
from cryptography.fernet import Fernet
import pyodbc
import pandas as pd
import shutil
import time
from datetime import datetime, timedelta
from IPython.display import display, HTML
from sqlalchemy import create_engine, NVARCHAR, text
import sqlalchemy
import urllib
from sqlalchemy.types import String, Integer, DateTime  # Adicione esta linha no início do seu script

# Função para destacar as mensagens de erro e funcionamento
def print_error(*args):
    message = ' '.join(map(str, args))
    display(HTML(f"<span style='color:red;'>{message}</span>"))


def print_ok(*args):
    message = ' '.join(map(str, args))
    display(HTML(f"<span style='color:green;'>{message}</span>"))

Formato do arquivo config.txt:

- caminho=/caminho/para/a/pasta
- instancia=sua_instancia_db
- banco=seu_banco_de_dados
- login=seu_login
- senha=sua_senha_criptografada


## 1 - Abrir o Bloco de Notas e Carregar as Variáveis

In [5]:
def load_config(file_path):
    config = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line and '=' in line:  # Ignora linhas em branco e verifica se contém '='
                try:
                    key, value = line.split('=', 1)  # Dividir apenas na primeira '='
                    config[key] = value
                except ValueError:
                    print_error(f"Ignorando linha inválida no arquivo de configuração: {line}")
    print_ok(f"1 - Configurações carregadas.")
    return config

## 2 - Carregar uma Coleção de Todos os Arquivos Excel na Pasta

In [8]:
def load_excel_files(folder_path):
    # Carrega todos os caminhos dos arquivos .xls da pasta especificada
    excel_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xls')]
    
    # Exibe os arquivos encontrados
    if excel_files:
        print(f"Arquivos Excel encontrados: {excel_files}")
    else:
        print_error("Nenhum arquivo Excel encontrado na pasta.")
    
    return excel_files

## 3 - Descriptografar a Senha

In [9]:
def decrypt_password(encrypted_password):
    # Chave usada para a descriptografia (substitua pela sua chave real)
    key = b''
    cipher_suite = Fernet(key)

    print("3 - Iniciando a descriptografia da senha...")

    try:
        # Descriptografa a senha
        decrypted_password = cipher_suite.decrypt(encrypted_password.encode()).decode()
        print_ok("3 - Senha descriptografada com sucesso!")
    except Exception as e:
        print_error(f"3 - Erro durante a descriptografia: {e}")
        return None

    return decrypted_password

## 4 - Conectar com o DBMimport

In [14]:
def connect_to_db(instance, database, username, password):
    print("4 - Iniciando a conexão com o banco de dados...")
    
    # Monta a string de conexão
    params = urllib.parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={instance};DATABASE={database};UID={username};PWD={password}')
    connection_string = f'mssql+pyodbc:///?odbc_connect={params}'
    
    try:
        # Tenta conectar ao banco de dados
        engine = create_engine(connection_string)
        connection = engine.connect()
        print_ok("4 - Conexão estabelecida com sucesso!")
        return connection
    except Exception as e:
        print_error(f"4 - Erro ao conectar ao banco de dados: {e}")
        return None

## 5 - Ler a Coleção e Inserir os Registros Não Nulos na tabela

In [17]:
def insert_records_to_db(file_paths, connection):
    # Nome fixo da tabela
    table_name = ''

    # Processa os dados do Excel e obtém um DataFrame
    df = process_excel_data(file_paths)

    try:
        with connection.begin():  # Inicia uma nova transação
            # Verifica se a tabela existe
            exists = connection.execute(
                text(f"""
                SELECT COUNT(*) 
                FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_NAME = '{table_name}'
                """)
            ).scalar()

            if not exists:
                print_error(f"A tabela {table_name} não existe. Certifique-se de que a tabela esteja criada antes de inserir os dados.")
                return
            
            print_ok(f"Tabela {table_name} existe.")

            # Inserir os dados
            for index, row in df.iterrows():
                connection.execute(
                    text(f"""
                    INSERT INTO {table_name} ({', '.join(df.columns)})
                    VALUES ({', '.join([':' + col for col in df.columns])})
                    """),
                    row.to_dict()  # Converte a linha em um dicionário para os parâmetros
                )

        print_ok(f"Registros inseridos com sucesso na tabela {table_name}.")
        return True
    except Exception as e:
        print_error(f"Erro ao inserir registros: {str(e)}")
        return False

## 6 - Verificar se Existe a Pasta "Processados" e Criar se Não Existir

In [20]:
def create_processed_folder(folder_path):
    processed_folder = os.path.join(folder_path, 'Processados')
    
    print(f"6 - Verificando a existência da pasta: {processed_folder}...")
    
    if not os.path.exists(processed_folder):
        print("6 - Pasta 'Processados' não encontrada. Criando a pasta...")
        os.makedirs(processed_folder)
        print("6 - Pasta 'Processados' criada com sucesso!")
    else:
        print("6 - A pasta 'Processados' já existe.")
        
    return processed_folder

## 7 - Mover as Planilhas Carregadas para a Pasta "Processados"

In [23]:
def move_to_processed(file_path, processed_folder):
    print(f"7 - Movendo o arquivo: {file_path} para a pasta 'Processados'...")
    
    try:
        shutil.move(file_path, processed_folder)
        print_ok(f"7 - Arquivo movido com sucesso para: {processed_folder}")
    except Exception as e:
        print_error(f"7 - Erro ao mover o arquivo: {e}")

## 8 - O Robô Deve Dormir Somente Depois que o Caminho Raiz Estiver Vazio de Planilhas Excel

In [26]:
def wait_until_empty(folder_path):
    print(f"8 - Aguardando até que a pasta {folder_path} fique vazia...")
    
    while len(load_excel_files(folder_path)) > 0:
        print("8 - A pasta ainda contém arquivos Excel. Aguardando 1 minuto...")
        time.sleep(60)  # Espera 1 minuto
    
    print_ok("8 - A pasta está vazia. Continuando o processo.")

## 9 - Colocar o Timer para o Robô Acordar às 9 da Manhã do Dia Seguinte

In [39]:
def sleep_until_next_run():
    global ultima_execucao
    # Definindo os horários de início e término do trabalho com horas e minutos
    horario_inicio = (9, 0)  # (HORAS, MINUTOS)
    horario_termino = (12, 00)  # (HORAS, MINUTOS)
    
    # Definindo o intervalo entre as tentativas em minutos
    horario_tentativas = 30  # Intervalo entre as tentativas ( EM MINUTOS )

    now = datetime.now()
    print(f"9 - Data e hora atuais: {now}")

    if ultima_execucao > 0:
        # Se já executou hoje, dorme até o próximo dia útil no horário de início
        if now.weekday() >= 5:  # Fim de semana (sábado ou domingo)
            next_run_time = (now + timedelta(days=(7 - now.weekday()))).replace(hour=horario_inicio[0], minute=horario_inicio[1], second=0, microsecond=0)
            print_ok(f"9 - Fim de semana. Dormindo até segunda-feira: {next_run_time}")
        else:
            next_run_time = (now + timedelta(days=1)).replace(hour=horario_inicio[0], minute=horario_inicio[1], second=0, microsecond=0)
            print_ok(f"9 - Já trabalhou hoje. Dormindo até o dia seguinte às {horario_inicio[0]}h{horario_inicio[1]:02d}: {next_run_time}")
    else:
        # Se ainda não executou hoje, verifica o horário e dorme pelo intervalo definido
        if now.weekday() >= 5:  # Fim de semana
            next_run_time = (now + timedelta(days=(7 - now.weekday()))).replace(hour=horario_inicio[0], minute=horario_inicio[1], second=0, microsecond=0)
            print_ok(f"9 - Fim de semana. Dormindo até segunda-feira: {next_run_time}")
        else:
            if now.hour < horario_inicio[0] or (now.hour == horario_inicio[0] and now.minute < horario_inicio[1]):
                # Se for antes do horário de início, dorme até o horário de início
                next_run_time = now.replace(hour=horario_inicio[0], minute=horario_inicio[1], second=0, microsecond=0)
                print(f"9 - Aguardando até às {horario_inicio[0]}h{horario_inicio[1]:02d}: {next_run_time}")
            elif now.hour < horario_termino[0] or (now.hour == horario_termino[0] and now.minute < horario_termino[1]):
                # Se estiver dentro do horário de trabalho, dorme pelo intervalo definido
                next_run_time = now + timedelta(minutes=horario_tentativas)
                print(f"9 - Dentro do horário de trabalho. Dormindo por {horario_tentativas} minutos: {next_run_time}")
            else:
                # Se já passou do horário de trabalho, dorme até o próximo dia útil
                next_run_time = (now + timedelta(days=1)).replace(hour=horario_inicio[0], minute=horario_inicio[1], second=0, microsecond=0)
                print(f"9 - Já passou do horário de trabalho. Dormindo até o dia seguinte às {horario_inicio[0]}h{horario_inicio[1]:02d}: {next_run_time}")

    # Calcula o tempo de sono
    sleep_time = (next_run_time - now).total_seconds()
    hours, remainder = divmod(sleep_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Exibe o tempo de espera no formato horas:minutos:segundos
    print_ok(f"9 - Tempo de espera: {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}")
    time.sleep(sleep_time)

    ultima_execucao = 0

## 10 - Transformar o Excel em um DataFrame Pandas

In [42]:
def process_excel_data(file_paths):
    try:
        # Ler o arquivo Excel
        df = pd.read_excel(file_paths)

        # Remover colunas 'Unnamed' e linhas com valores nulos nas colunas obrigatórias
        non_null_records = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        required_columns = ['DATA', 'CODIGO', 'CENTRODECUSTO', 'POSTODESERVICO', 'FUNCIONARIO', 'TIPO']
        non_null_records = non_null_records.dropna(subset=required_columns)
        non_null_records = non_null_records.dropna(thresh=len(non_null_records.columns) - 10).copy()

        # Garantir que as colunas tenham os tipos corretos
        non_null_records['DATA'] = pd.to_datetime(non_null_records['DATA'], errors='coerce')
        non_null_records['CODIGO'] = non_null_records['CODIGO'].astype(str)
        non_null_records['CENTRODECUSTO'] = non_null_records['CENTRODECUSTO'].astype(str)
        non_null_records['POSTODESERVICO'] = non_null_records['POSTODESERVICO'].astype(str)
        non_null_records['FUNCIONARIO'] = non_null_records['FUNCIONARIO'].astype(str)
        non_null_records['TIPO'] = non_null_records['TIPO'].astype(str)
        non_null_records['Refeicao'] = non_null_records['Refeicao'].astype(str)
        non_null_records['Ausencia'] = non_null_records['Ausencia'].astype(str)
        non_null_records['DataReposicao'] = non_null_records['DESCRICAOALTERACAO'].fillna(0).astype('int32')
        non_null_records['OBSINSPETOR'] = non_null_records['OBSINSPETOR'].astype(str)
        non_null_records['ALTERACAO'] = non_null_records['ALTERACAO'].astype(str)
        non_null_records['OCORRENCIA'] = non_null_records['OCORRENCIA'].astype(str)
        non_null_records['DESCRICAOALTERACAO'] = non_null_records['DESCRICAOALTERACAO'].fillna(0).astype('int32')
        non_null_records['OBSSUPERVISOR'] = non_null_records['OBSSUPERVISOR'].astype(str)
        non_null_records['StatusSupervisor'] = non_null_records['StatusSupervisor'].astype(str)
        non_null_records['OBSCOORDENADOR'] = non_null_records['OBSCOORDENADOR'].fillna(0).astype('int32')
        non_null_records['StatusCoordenador'] = non_null_records['StatusCoordenador'].astype(str)

        # Tratar valores nulos: colunas numéricas com 0 e colunas de texto com string vazia
        for column in non_null_records.columns:
            if non_null_records[column].dtype in ['float64', 'int64']:
                non_null_records[column] = non_null_records[column].fillna(0)  # Preencher numéricos com 0

            else:
                non_null_records[column] = non_null_records[column].fillna('')  # Preencher strings vazias

        # # Lista de colunas a serem excluídas
        # columns_to_drop = ['DataReposicao']
        # # Exclui as colunas especificadas, se existirem
        # non_null_records = non_null_records.drop(columns=[col for col in columns_to_drop if col in non_null_records.columns], errors='ignore')
        
        # Exibir os tipos de dados e uma amostra dos dados para verificação
        print("10 - Tipos de dados das colunas:")
        print(non_null_records.dtypes)
        
        display(non_null_records)

        return non_null_records

    except Exception as e:
        print_error(f"10 - Ocorreu um erro: {e}")


## Main - Loop Principal para o Robô

In [45]:
def run_robot():
    global ultima_execucao
    while True:
        print_ok("Iniciando o robô...")
        # Verifica se está na hora de trabalhar
        sleep_until_next_run()
        
        print("Carregando as variáveis do arquivo de configuração...")
        # Carregar as variáveis
        config = load_config("config.txt")

        print("Criando a pasta Processados se não existir...")
        # Criar a pasta Processados
        processed_folder = create_processed_folder(config['caminho'])

        print("Carregando arquivos Excel da pasta:", config['caminho'])
        # Carregar arquivos Excel
        excel_files = load_excel_files(config['caminho'])

        if excel_files:
            print("Conectando ao banco de dados...")
            # Conectar ao banco de dados
            senha = decrypt_password(config['senha'])
            db_connection = connect_to_db(config['instancia'], config['banco'], config['login'], senha)
            
            # Processar arquivos
            for file in excel_files:
                print(f"Processando arquivo: {file}")
                if insert_records_to_db(file, db_connection):  # Insira os registros no banco de dados
                    print(f"Arquivo {file} inserido no banco de dados.")
                    move_to_processed(file, processed_folder)  # Mova o arquivo para a pasta Processados
                    print(f"Arquivo {file} movido para a pasta Processados.")
                    ultima_execucao += 1

            print("Aguardando até que a pasta esteja vazia...")
            # Esperar até que a pasta esteja vazia
            wait_until_empty(config['caminho'])

# Iniciar o Robô

In [ ]:
# Variável global para controlar a execução diária
ultima_execucao = 0

run_robot()